In [289]:
import pandas as pd
import requests
import numpy as np
import re
from glob import glob
import xmltodict
import jmespath
import json
import unicodedata
import zipfile
import os

"""
Fonction de recherche récursive pour le json, nécessaire parce que les interventions sont éparpillées un peu partout dans l'arborescence du fichier de manière désordonnée.
elle permet de trouver tous les chemins d'une certaine clé où qu'elle soit.
"""
def find_in_obj(obj, condition, path=None):

    if path is None:
        path = []    

    if isinstance(obj, list):
        for index, value in enumerate(obj):
            new_path = list(path)
            new_path.append(index)
            for result in find_in_obj(value, condition, path=new_path):
                yield result 

    if isinstance(obj, dict):
        for key, value in obj.items():
            new_path = list(path)
            new_path.append(key)
            for result in find_in_obj(value, condition, path=new_path):
                yield result 

            if condition == key:
                new_path = list(path)
                new_path.append(key)
                yield new_path 

#récupérer les données à partir des chemins obtenus avec la fonction précédente                
def retrieve_value(key_list, dct):
    subdict = dct
    for k in key_list:
        subdict = subdict[k]
    return subdict

# Téléchargement des données

In [292]:
r = requests.get('https://data.assemblee-nationale.fr/static/openData/repository/15/vp/syceronbrut/syseron.xml.zip')
print("downloaded")
with open("temp.zip","wb") as fd:
    fd.write(r.content)
    
with zipfile.ZipFile("temp.zip","r") as zip_ref:
    zip_ref.extractall("debats")

r = requests.get('https://data.assemblee-nationale.fr/static/openData/repository/15/amo/deputes_senateurs_ministres_legislature/AMO20_dep_sen_min_tous_mandats_et_organes_XV.json.zip')
print("downloaded")
with open("temp.zip","wb") as fd:
    fd.write(r.content)
    
with zipfile.ZipFile("temp.zip","r") as zip_ref:
    zip_ref.extractall("acteurs")

os.remove("temp.zip")

downloaded
downloaded


# Extraction des données brutes

## données débats

In [327]:
list_seances = [] 
files = glob("debats/xml/compteRendu/*.xml")

for n,file in enumerate(files):
    with open(file,"r") as fd:
        doc = xmltodict.parse(fd.read())

     #conversion du xml en json
    contenu = doc["compteRendu"]["contenu"]
    contenu = json.loads(json.dumps(contenu))

    #on commence par récupérer les infos sur les orateurs-contexte, càd ceux qui ont officiellement la parole, localisés sous la clé 'interExtraction'
    path_list = find_in_obj(contenu,"interExtraction")
    textes = [retrieve_value(i,contenu) for i in path_list]
    
    #jmespath est une librairie de parsing de json qui me permet de récupérer tous les champs que je veux.
    a = jmespath.search("[].[\"@id_acteur\",\"@nom_orateur\",paragraphe[].\"@id_syceron\"]",textes)
    if len(a)>0:
        interextraction = pd.DataFrame(a).explode(2)
        interextraction.columns = ["orateur_contexte_id","orateur_contexte","id_intervention"]
        interex =True
    else:
        interex = False

    #on récupére les infos et le texte de l'intervention proprement drite localisée sous la clé 'paragraphe'
    path_list = find_in_obj(contenu,"paragraphe")
    textes = [retrieve_value(i,contenu) for i in path_list]
    a = jmespath.search('[].[\"@id_syceron\",orateurs.orateur.nom,orateurs.orateur.id,orateurs.orateur.qualite, \"@code_grammaire\",texte, texte.\"#text\", texte.italique]',textes)
    df = pd.DataFrame(a)
    
    #le texte est parfois dans une string, parfois dans un dictionnaire, je créée une colonne unique à partir des deux
    df[6] = np.where(df[6].isna(),df[5],df[6])
    df.columns = ["id_intervention","orateur","id_orateur","qualité","code_intervention","to_drop","texte","italique"]
    
    #je merge avec les données des orateurs-contexte
    if interex:
        df= interextraction.merge(df,how="right",on="id_intervention")
    df = df.drop(["to_drop"],axis=1).dropna(subset=["orateur"])
    df["intervention_ordre"] = [i+1 for i in range(len(df))]
    
    metadonnes = doc["compteRendu"]["metadonnees"]
    df["date"] = metadonnes["dateSeance"][:8]
    df["seance"] = metadonnes["numSeanceJour"]
    df["session"] = metadonnes["session"]

    list_seances.append(df)
    if n%10 == 0:
        print(n,end=",")
print("making df")
df = pd.concat(list_seances)


0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300,310,320,330,340,350,360,370,380,390,400,410,420,430,440,450,460,470,480,490,500,510,520,530,540,550,560,570,580,590,600,610,620,630,640,650,660,670,680,690,700,710,720,730,740,750,760,770,780,790,800,810,820,830,840,850,860,870,880,890,900,910,920,930,940,950,960,970,980,990,1000,1010,1020,1030,1040,1050,1060,1070,1080,1090,1100,1110,1120,1130,1140,1150,1160,1170,1180,1190,1200,1210,1220,1230,1240,1250,1260,1270,1280,1290,1300,1310,1320,1330,1340,1350,1360,1370,1380,1390,1400,1410,1420,1430,1440,1450,making df


## Nettoyage

In [329]:
#il y a parfois des dictionnaires vides dans la colonne texte, je les enlève
print("treating text")
df.loc[df["texte"].apply(lambda x : type(x)) == dict,"texte"] = None

#il y a parfois des valeurs None dans les italiques; je les enlève.
print("treating ital")
df["italique"] = df["italique"].apply(lambda x : [i for i in x if i] if isinstance(x,list) else x)

df["date"] = pd.to_datetime(df["date"])

#nettoyage des noms des orateurs
df["orateur"] = df["orateur"].str.replace("\(.+","").str.strip().str.strip(",")

df = df.sort_values(["orateur",'id_orateur'])

#les id orateur sont parfois manquante, je les remplis
df["id_orateur"] = df.groupby("orateur")["id_orateur"].fillna(method="ffill")
df["id_orateur"] = "PA" + df["id_orateur"]

#un même orateur a parfois de multiples id, je les remplace par l'id la plus commune
correct_id = df.groupby('orateur')["id_orateur"].agg(pd.Series.mode).astype(str).replace({"[]" : "PA0"})
correct_id.loc[correct_id.str.contains("\[").fillna(False)] = "PAO"
a = df.merge(correct_id,left_on="orateur",right_index=True, how="left")["id_orateur_y"]
df["id_orateur"] = a

#nettoyage du texte
for col in ["orateur_contexte","orateur","texte"]:
    df[col] = df[col].fillna("").apply(lambda x : unicodedata.normalize("NFKD",x))


treating text
treating ital


<ipython-input-329-b2af22642605>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df["orateur"] = df["orateur"].str.replace("\(.+","").str.strip().str.strip(",")


## Données appartenance politique

In [330]:
#récupération des acteurs
files = glob("acteurs/json/acteur/*.json")
acteur_liste = []
for file in files:
    with open(file,"r") as fd:
        d = json.load(fd)["acteur"]["mandats"]["mandat"]
        
    d = pd.json_normalize(d)
    d = d[d["typeOrgane"] == "PARPOL"]
    acteur_liste.append(d)
    
df_mandats = pd.concat(acteur_liste)
df_mandats = df_mandats[["acteurRef","dateDebut","dateFin","organes.organeRef"]]

#récupération des organes d'appartenance des acteurs
orgs = df_mandats["organes.organeRef"].unique()
orgs_liste = []

for org in orgs:
    file = "acteurs/json/organe/" + org +  ".json"
    with open(file,"r") as fd:
        f = json.load(fd)
        orgs_liste.append(f)
df_partis = pd.json_normalize(orgs_liste)
df_partis = df_partis[["organe.uid","organe.libelle","organe.libelleAbrev"]]

#merge acteurs-organes
df_mandats= df_mandats.merge(df_partis,left_on="organes.organeRef",right_on="organe.uid")[["acteurRef","organes.organeRef","organe.libelle","dateDebut","dateFin"]]
df_mandats.columns = ["id_orateur","id_parti","parti_label","debut","fin"]

df_mandats.loc[df_mandats["fin"].isna(),"fin"] = pd.to_datetime("2099-01-01")

for col in ("debut",'fin'):
    df_mandats[col] = pd.to_datetime(df_mandats[col])
print("done")

done


# Merge des données

## Données orateur

In [331]:
files = glob("acteurs/json/acteur/*.json")
acteurs = [i.replace("acteurs/json/acteur/","").replace(".json","").strip() for i in files]

#on merge uniquement pour les acteurs qui ont un id. Les ministres ou intervenants extérieurs n'en ont pas par exemple.
df_no_id = df[(~df["id_orateur"].isin(acteurs))]
df_id = df[(df["id_orateur"].isin(acteurs))]

df_id = df_id.merge(df_mandats,on="id_orateur",how="left")
#les groupes politiques ne sont définis qu'à partir de décembre 2017, je les avance au début de la législature
df_id.loc[df_id["debut"] == "2017-12-01",'debut'] = pd.to_datetime("2017-06-20")

#je sélectionne le mandat qui correspond à la date de l'intervention (un député peut avoir changer de groupe au cours de la législature)
df_id["mandat_ok"] = (df_id["date"] >= df_id["debut"]) & (df_id["date"] <= df_id["fin"])
df_id2 = df_id.sort_values(["id_intervention","mandat_ok"],ascending=False).drop_duplicates("id_intervention").drop(["id_parti","debut","fin","mandat_ok"],axis=1)

df = pd.concat([df_no_id,df_id2])

df["parti_label"] = df["parti_label"].replace({"En marche !" : "La République en Marche", "Front National" : "Rassemblement national"})
df = df.rename({"parti_label" : "parti_orateur"},axis=1)

print("done")

done


## Données orateurs contexte

In [332]:
#exactement la même chose mais pour l'orateur contexte
df_no_id = df[(~df["orateur_contexte_id"].isin(acteurs))]
df_id = df[(df["orateur_contexte_id"].isin(acteurs))]

df_mandats = df_mandats.rename({"id_orateur" : "orateur_contexte_id"},axis=1)

df_id = df_id.merge(df_mandats,on="orateur_contexte_id",how="left")
df_id.loc[df_id["debut"] == "2017-12-01",'debut'] = pd.to_datetime("2017-06-20")

df_id["mandat_ok"] = (df_id["date"] >= df_id["debut"]) & (df_id["date"] <= df_id["fin"])
df_id2 = df_id.sort_values(["id_intervention","mandat_ok"],ascending=False).drop_duplicates("id_intervention").drop(["id_parti","debut","fin","mandat_ok"],axis=1)

df = pd.concat([df_no_id,df_id2])

df["parti_label"] = df["parti_label"].replace({"En marche !" : "La République en Marche", "Front National" : "Rassemblement national"})
df = df.rename({"parti_label" : "parti_orateur_contexte"},axis=1)

df["id_orateur"] = df["id_orateur"].replace({"PA0" : np.nan})
print("done")

done


# Sauvegarde des données

In [336]:
ordered_columns = ["id_intervention","date","seance","session","intervention_ordre","orateur","id_orateur","parti_orateur","code_intervention","qualité","texte","italique","orateur_contexte","orateur_contexte_id","parti_orateur_contexte"]
df = df[ordered_columns].sort_values(["date","seance","intervention_ordre"])
df.to_csv("debats_assemblee.csv.zip",index=False)
df.head(10)

,id_intervention,date,seance,session,intervention_ordre,orateur,id_orateur,parti_orateur,code_intervention,qualité,texte,italique,orateur_contexte,orateur_contexte_id,parti_orateur_contexte
541710,980116,2017-06-27,Unique,Session ordinaire 2016-2017,1,M. le président,PA721824,La République en Marche,OUV_SEAN_2_1,None,La séance est ouverte.,None,,NaN,NaN
541712,980120,2017-06-27,Unique,Session ordinaire 2016-2017,2,M. le président,PA721824,La République en Marche,ANNONCE_CCATION_1_10,None,Je déclare ouverte la XVlégislature de l’Ass...,None,,NaN,NaN
541714,980123,2017-06-27,Unique,Session ordinaire 2016-2017,3,M. le président,PA721824,La République en Marche,ANNONCE_CCATION_1_10,None,"Aux termes de l’article 1 du règlement, les s...",None,,NaN,NaN
541716,980126,2017-06-27,Unique,Session ordinaire 2016-2017,4,M. le président,PA721824,La République en Marche,ANNONCE_CCATION_1_10,None,En application de l’article L.O. 179 du code e...,None,,NaN,NaN
541718,980129,2017-06-27,Unique,Session ordinaire 2016-2017,5,M. le président,PA721824,La République en Marche,ANNONCE_CCATION_1_10,None,Les décrets relatifs à la composition du Gou...,Journal officiel,,NaN,NaN
541720,980132,2017-06-27,Unique,Session ordinaire 2016-2017,6,M. le président,PA721824,La République en Marche,ANNONCE_CCATION_1_10,None,J’ai le regret de porter à la connaissance de...,None,,NaN,NaN
541722,980135,2017-06-27,Unique,Session ordinaire 2016-2017,7,M. le président,PA721824,La République en Marche,ANNONCE_CCATION_1_10,None,Monsieur le secrétaire d’État chargé des re...,(Applaudissements.),,NaN,NaN
541724,980142,2017-06-27,Unique,Session ordinaire 2016-2017,8,M. le président,PA721824,La République en Marche,ELECTION_PDT_1_10,None,"L’ordre du jour appelle, conformément à l’ar...",None,,NaN,NaN
541726,980144,2017-06-27,Unique,Session ordinaire 2016-2017,9,M. le président,PA721824,La République en Marche,ELECTION_PDT_1_30,None,Sont désignés scrutateurs titulaires : M. Br...,None,,NaN,NaN
541728,980146,2017-06-27,Unique,Session ordinaire 2016-2017,10,M. le président,PA721824,La République en Marche,ELECTION_PDT_1_50,None,Le sort désigne la lettre U.Pour faciliter le...,None,,NaN,NaN
